## SDG indicator 11.3.1: Ratio of land consumption rate to population growth rate

- #### import datasets: 
     - #### mid-year opulation estimates from ONS for latest year (e.g 2019) and previous year (3 years before)
     - #### land area datasets from OS for latest year (e.g 2019) and previous year (3 years before)
- #### check LAD boundary changes between the 2year periods
- #### calculate population growth rate
- #### calculate land consumption rate
- #### calculate Ratio of land consumption rate to population growth rate

In [76]:
# import libraries
from pathlib import Path
import geopandas as gpd
import pandas as pd
from pandas import DataFrame, merge
import numpy as np
import matplotlib.pyplot as plt
import shapely
import csv
import math
import os
# print(Path('.').resolve())

In [77]:
#calculate Population Growth rate (PGR) = LN(Pop_t+n/Pop_t )/(y)
     # Where: LN is the natural logarithm value
         # Pop_t is the total population within the urban area/city in the past/initial year
         # Pop_t+n is the total population within the urban area/city in the current/final year
         # y is the number of years between the two measurement periods

# read population data 
    # make sure all columns with numbers have float dtype by using (df = pd.read_csv(<PATH>, thousands=',')
pop_est16ew = pd.read_csv(r'R:\SDGs\SDG_11_3_1\data\population\SAPE20DT1_mid2016_lsoa_ew.csv',thousands=',').set_index('LSOA11CD')
pop_est19ew = pd.read_csv(r'R:\SDGs\SDG_11_3_1\data\population\SAPE22DT2_mid2019_lsoa_ew.csv',thousands=',').set_index('LSOA11CD')
pop_est16sc = pd.read_csv(r'R:\SDGs\SDG_11_3_1\data\population\SAPE16_mid2016_dz_sc.csv',thousands=',').set_index('DZ2011CD')
pop_est19sc = pd.read_csv(r'R:\SDGs\SDG_11_3_1\data\population\SAPE19_mid2019_dz_sc.csv',thousands=',').set_index('DZ2011CD')

    # remove whitespace before and after column names
pop_est16ew.columns = pop_est16ew.columns.str.strip()
pop_est19ew.columns = pop_est19ew.columns.str.strip()
pop_est16sc.columns = pop_est16sc.columns.str.strip()
pop_est19sc.columns = pop_est19sc.columns.str.strip()

display(pop_est16ew.head())
display(pop_est19ew.head())
display(pop_est16sc.head())
display(pop_est19sc.head())


    # merge population tables for england and wales and for scotland for 2016 and 2019
pop_est19_16ew = pd.merge(pop_est19ew, pop_est16ew, on='LSOA11CD', how='outer', indicator=True, suffixes=('_19','_16'))
pop_est19_16sc = pd.merge(pop_est19sc, pop_est16sc, on='DZ2011CD', how='outer', indicator=True, suffixes=('_19','_16'))

    # change data type from string to float for columns with population data
#pop_est19_16ew['All_Ages_2016']=pop_est19_16ew['All_Ages_2016'].str.replace(',','').astype(np.float32)
#pop_est19_16ew['All_Ages_2019']=pop_est19_16ew['All_Ages_2019'].str.replace(',','').astype(np.float32)
#pop_est19_16sc['All_Ages_2016']=pop_est19_16sc['All_Ages_2016'].str.replace(',','').astype(np.float32)
#pop_est19_16sc['All_Ages_2019']=pop_est19_16sc['All_Ages_2019'].str.replace(',','').astype(np.float32)

    #check if merge operation has run successfully
display(pop_est19_16ew.head())
display(pop_est19_16sc.head())

#calculate Population Growth rate (PGR) = LN(Pop_t+n/Pop_t )/(y)

    #create new column (pgr19_16) and calculate PGR
pop_est19_16ew['pgr19_16'] = np.log(pop_est19_16ew['All_Ages_2019']/pop_est19_16ew['All_Ages_2016'])/3
pop_est19_16sc['pgr19_16'] = np.log(pop_est19_16sc['All_Ages_2019']/pop_est19_16sc['All_Ages_2016'])/3

    #check if code has run successfully
display(pop_est19_16ew['pgr19_16'].head())
display(pop_est19_16sc['pgr19_16'].head())

    # write table with pgr calculation to file
pop_est19_16ew.to_csv(r'R:\SDGs\SDG_11_3_1\intermediate\pgr_19_16ew.csv')
pop_est19_16sc.to_csv(r'R:\SDGs\SDG_11_3_1\intermediate\pgr_19_16sc.csv')

    




FileNotFoundError: [Errno 2] File R:\SDGs\SDG_11_3_1\data\population\SAPE20DT1_mid2016_lsoa_ew.csv does not exist: 'R:\\SDGs\\SDG_11_3_1\\data\\population\\SAPE20DT1_mid2016_lsoa_ew.csv'

In [ ]:
# calculate Land Consumption Rate (LCR) = ((Vpresent - Vpast)/Vpast)/(t)
     # where : Vpresent is total built-up area in current year
            # Vpast is total built-up area in past year
            # (t) is number of years between Vpresnt and Vpast (or length in years of the period considered)

# read land_area data 
    # make sure all columns with numbers should have float dtype by using (df = pd.read_csv(<PATH>, thousands=',')  
land_area16gb = pd.read_csv(r'R:\SDGs\SDG_11_3_1\data\OS\lsoa2016_landcover_area_GB.csv', thousands=',')
land_area19_sc = pd.read_csv(r'R:\SDGs\SDG_11_3_1\data\OS\lsoa2019_landcover_area_sc.csv', thousands=',')
land_area19_ew = pd.read_csv(r'R:\SDGs\SDG_11_3_1\data\OS\lsoa2019_landcover_area_ew.csv', thousands=',')


    # clean columns names by removing whitespaces before and afer column names
land_area16gb.columns = land_area16gb.columns.str.strip()
land_area19_sc.columns = land_area19_sc.columns.str.strip()
land_area19_ew.columns = land_area19_ew.columns.str.strip()

    #inspect imported data visually
display(land_area16gb.head())
display(land_area19_sc.head())
display(land_area19_ew.head())

    #split land_area file for 2016 into England and Wales (EW) and Scotland (SC) (only necessary step because OS data is mixed between OAs for SC and LSOAs for EW)
filt_ew=(land_area16gb['CTRY']=='E') | (land_area16gb['CTRY']=='W') # filter to separate EW data
filt_sc=(land_area16gb['CTRY']=='S') # # filter to separate SC data

land_area16_ew=land_area16gb.loc[filt_ew]
land_area16_sc=land_area16gb.loc[filt_sc]

    #rename SC column names to SC format 
land_area16_sc.rename({'LSOA11CD':'DataZone2011Code','LSOA11NM':'DataZone2011Name','LAD16CD':'CouncilArea2016Code','LAD16NM':'CouncilArea2016Name'}, axis=1, inplace=True)
    #write 2016 land_area files to file
land_area16_ew.to_csv(r'R:\SDGs\SDG_11_3_1\intermediate\land_area16_ew.csv')
land_area16_sc.to_csv(r'R:\SDGs\SDG_11_3_1\intermediate\land_area16_sc.csv')

display(land_area16_ew.head())
display(land_area16_sc.head())

    #filter landcover type = manmade for 2019 data
filt_19ewmm = (land_area19_ew['landcover_type']=='Manmade') # filter 'manmade' landcover for EW
land_area19_ewmm = land_area19_ew.loc[filt_19ewmm]
land_area19_ewmm.to_csv(r'R:\SDGs\SDG_11_3_1\intermediate\land_area19_ewmm.csv')

filt_19scmm = (land_area19_sc['landcover_type']=='Manmade') # filter 'manmade' landcover for SC
land_area19_scmm = land_area19_sc[['CTRY','DataZone2011Code','CouncilArea2011Code','landcover_type','area_2019']].loc[filt_19scmm]
   # aggregate Scotland OAs to DZs (equavalent of LSOA in EW)
land_area19_scdzmm = land_area19_scmm.groupby(['CouncilArea2011Code','DataZone2011Code','landcover_type']).sum()
land_area19_scdzmm.to_csv(r'R:\SDGs\SDG_11_3_1\intermediate\land_area19_scdzmm.csv') #write to csv

display(land_area19_ewmm.head()) # EW LSOA 'manmade' land_area 
display(land_area19_scdzmm.head()) # EW DZ 'manmade' land_area

# calculate Land Consumption Rate (LCR) = ((Vpresent - Vpast)/Vpast)/(t)
      
    #read land_area files and merge to calculate LCR
land_area16_ew = pd.read_csv(r'R:\SDGs\SDG_11_3_1\intermediate\land_area16_ew.csv').set_index('LSOA11CD')
land_area16_sc = pd.read_csv(r'R:\SDGs\SDG_11_3_1\intermediate\land_area16_sc.csv').set_index('DataZone2011Code')
land_area19_scdzmm = pd.read_csv(r'R:\SDGs\SDG_11_3_1\intermediate\land_area19_scdzmm.csv').set_index('DataZone2011Code')
land_area19_ewmm = pd.read_csv(r'R:\SDGs\SDG_11_3_1\intermediate\land_area19_ewmm.csv').set_index('LSOA11CD')
                             
    # merge 2019 and 2016 land_area files for EW and SC to create one table for each 
land_area19_16ewmm = pd.merge(land_area19_ewmm, land_area16_ew, on='LSOA11CD', how='outer', indicator=True, suffixes=('_19','_16'))
land_area19_16scmm = pd.merge(land_area19_scdzmm, land_area16_sc, on='DataZone2011Code', how='outer', indicator=True, suffixes=('_19','_16'))
   #check if code has run successfully
#display(land_area19_16ewmm.head())
#display(land_area19_16scmm.head())
                                                                   
    #write merge output to file
land_area19_16ewmm.to_csv(r'R:\SDGs\SDG_11_3_1\intermediate\land_area19_16ewmm.csv')
land_area19_16scmm.to_csv(r'R:\SDGs\SDG_11_3_1\intermediate\land_area19_16scmm.csv')
 
    #check and remove unwanted columns
#land_area19_16ewmm.columns
#land_area19_16scmm.columns




In [ ]:
   #check and remove unwanted columns
display(land_area19_16ewmm.columns)
display(land_area19_16scmm.columns)

In [ ]:

#calculate Land Consumption Rate (LCR) = ((Vpresent - Vpast)/Vpast)/(t) 
lcr19_16ewmm['LCR2016_19'] = (lcr19_16ewmm['area_2019']-lcr19_16ewmm['area_2016'])/3
lcr19_16scmm['LCR2016_19'] = (lcr19_16scmm['area_2019']-lcr19_16scmm['area_2016'])/3      
                                                                   
display(lcr19_16ewmm[LCR2016_19].head())   
display(lcr19_16escmm[LCR2016_19].head())

In [ ]:
# calculate Ratio of land consumption rate to population growth rate (LCRPGR)

            # LCRPGR = (Land Consumption Rate)/(Population Growth Rate)
    
# read population (PGR) data files and land_area (LCR) data files
pop_est19_16ew = pd.read_csv(r'R:\SDGs\SDG_11_3_1\intermediate\pgr_19_16ew.csv')
pop_est19_16sc = pd.read_csv(r'R:\SDGs\SDG_11_3_1\intermediate\pgr_19_16sc.csv')
land_area19_16ewmm = pd.read_csv(r'R:\SDGs\SDG_11_3_1\intermediate\land_area19_16ewmm.csv')
land_area19_16scmm = pd.read_csv(r'R:\SDGs\SDG_11_3_1\intermediate\land_area19_16scmm.csv')

In [ ]:
# import LAD boundaries for 2016 and 2019
ladboundaries = [r'R:\SDGs\SDG_11_3_1\data\lad\LAD_DEC_2019_UK_BFE.shp',r'R:\SDGs\SDG_11_3_1\data\lad\Local_Authority_Districts__December_2016__GB_BFE.shp']
for lad in ladboundaries:
    lad_gdf = gpd.read_file(lad)
    display(lad_gdf.head()) # check if code has run
    
    
    #plot lad bundaries
    ladsplot = lad_gdf.plot(
    color='none', edgecolor='red')
    plt.plot()
    